In [1]:
import numpy as np
import gurobipy as grb

In [2]:
def generateData(n):
    v = np.random.uniform(0, 100, size=(n))
    w = np.random.uniform(0, 100, size=(n))
    b = np.sum(w) * 0.2
    return v, w, b

In [3]:
v, w, b = generateData(100)

for x in [v, w, b]:
    print(x)

[97.28870363 15.98053637 36.35050858 36.41875693 26.58960581 21.07338935
 53.95211819 73.93396436 40.74194222 15.55469619 23.27249039 14.43970712
  7.46854656 74.85613985 38.62156682 53.29694061 54.74757366 66.92010433
 59.0247228  42.79866317 81.12291298 29.30210441 55.88029176 14.38042524
 99.22858632 60.50706568 82.68494971 20.93082178 22.08208713 11.97374967
 50.4142563  45.35544781 12.68760532 85.54189317 88.56275409 71.06934688
 59.40681408 19.46574276 47.3541541  92.57785519 42.21422344 10.03257346
 63.70542912  9.96052989 85.947174   80.88315239  5.73108589 66.41453397
 83.26830838 85.16514859 79.73057978 33.33084525 40.92676137 42.81565559
 46.10713953  7.80064343 41.11350843 21.24939224  1.7918351  68.95586089
 72.54136171 98.09691392 96.75519934 64.10595121 27.68642279 83.89468941
  4.51965826 22.43676923 62.77175275 35.06198681 61.52506483 36.23485897
 27.39082022 53.99618208 96.50985109 33.55858801 50.43250887 85.69759013
 43.2496651  70.1415181  72.59672732 38.68786245 83

In [19]:
def gurobiKP(values, weights, budget, verbose=0) -> tuple[dict, float]:
    """
    Returns the selected items and objective value 
    after solving the IP directly with Gurobi.
    """

    n = len(values)

    model = grb.Model("0-1 Knapsack")
    if not verbose: model.Params.OutputFlag = 0
    
    # Variables
    x = model.addVars(n, vtype=grb.GRB.BINARY)
    
    # Constraints
    model.addConstr(grb.quicksum(weights[i] * x[i] for i in range(n)) <= budget)
    model.addConstr(grb.quicksum(x[i] for i in range(n)) == 4)
    
    # Objective
    model.setObjective(grb.quicksum(values[i] * x[i] for i in range(n)), sense=grb.GRB.MAXIMIZE)
    
    model.optimize()
    
    solution = {i for i in range(n) if x[i].x > 0}
    return solution, model.ObjVal
    

In [5]:
x, z = gurobiKP(v, w, b)
print(x, z)

Academic license - for non-commercial use only - expires 2022-08-07
Using license file C:\Users\Simanta\gurobi.lic
{1, 2, 6, 7, 13, 14, 15, 16, 17, 22, 24, 25, 30, 33, 34, 35, 39, 44, 48, 52, 53, 59, 60, 61, 62, 65, 68, 70, 73, 74, 75, 77, 78, 82, 83, 84, 86, 95, 96} 2655.3849444271505


In [6]:
def getCover(weight, budget, x_bar, verbose=0) -> tuple[dict, float]:
    """Returns the cover and the objective value"""
    

    n = len(weight)
    
    model = grb.Model("Cover")
    if not verbose: model.Params.OutputFlag = 0

    # Variables
    z = model.addVars(n, vtype=grb.GRB.BINARY)

    # Constraints
    model.addConstr(grb.quicksum(weight[i] * z[i] for i in range(n)) >= budget + 1)
    
    # Objective
    model.setObjective(grb.quicksum((1 - x_bar[i]) * z[i] for i in range(n)))
    
    # Solve
    model.optimize()
    
    solution = {i for i in range(n) if z[i].x > 0}
    return solution, model.ObjVal
    

In [7]:
x_bar = {i: np.random.uniform(0, 1) for i, _ in enumerate(w)}
x, z = getCover(w, b, x_bar)

print(x)

{89, 98, 5, 10, 42, 74, 50, 24, 57, 26, 28, 94, 63}


In [8]:
def lift(cover: dict, alphas: dict, t: int, weights: float, budget: float, verbose=0) -> int:
    """
    Returns alpha for the variable for the given t 
    after lifting the variable for the given t, cover and the KP instance.
    """

    if t in cover:
        return 1
    elif t in alphas:    # If already lifted
        return alphas[t]
    
    n = len(values)
    
    # Case 1: x[t] = 0, inequality created by the input cover and alphas is valid
    # Nothing to do
    
    # Case 2: x[t] = 1
    model = grb.Model("Lifting")
    if not verbose: model.Params.OutputFlag = 0
    
    # Variables with x[t] set to 1, for other indeces binary variables are created.
    x = {i: model.addVar(vtype=grb.GRB.BINARY, name=f'x_{i}') for i in range(n) if i != t}
    
    # Constraints
    model.addConstr(
        grb.quicksum(weights[i] * x[i] for i in alphas)
        + grb.quicksum(weights[i] * x[i] for i in cover) <= budget - weights[t]
    )
    
    # Objective
    model.setObjective(
        grb.quicksum(alpha_i * x[i] for i, alpha_i in alphas.items())
        + grb.quicksum(x[i] for i in cover)
    )
    
    if verbose:
        print('Constraints: ' 
            + ' + '.join(f'{weights[i]} * x_{i+1}' for i in alphas) 
            + ' + ' + ' + '.join(f'{weights[i]} * x_{i+1}' for i in cover) 
            + f' <= {budget} - {weights[t]}')
    
    # print(model.getObjective())
    
    model.ModelSense = grb.GRB.MAXIMIZE
    
    # Solve
    model.optimize()
    
    # Get the alpha for index t
    zeta_t = model.ObjVal
    alpha = len(cover) - 1 - zeta_t
    
    return alpha if alpha >= 0 else 0
    

In [9]:
# a = [12, 9, 7, 5, 5, 3]
# budget = 14
# x_bar = {i: 0 for i, _ in enumerate(a)}

# c, z = getCover(a, budget, x_bar)
# alphas = {}
# cc = {2, 4, 5}

# a1 = lift(cc, alphas, 0, a, budget)
# print(a1)

# alphas[0] = int(a1)
# a2 = lift(cc, alphas, 1, a, budget)
# print(a2)


# alphas[1] = a2
# a3 = lift(cc, alphas, 3, a, budget)
# print(a3)


In [22]:
from itertools import permutations

class HashableDict(dict):
    def __hash__(self):
        return hash(frozenset(self))


# Original
v = v#[10, 10, 10, 10, 10, 10]
a = w#[12, 9, 7, 5, 5, 3]
n = len(a)
budget = b#14
ineqs = set()
fd_ineqs = set()


model = grb.Model("init-model")
model.Params.OutputFlag = 0

x = model.addVars(len(a), lb=0, ub=1, vtype=grb.GRB.CONTINUOUS)

model.addConstr(grb.quicksum(a[i] * x[i] for i in range(len(a))) <= budget)
model.addConstr(grb.quicksum(x[i] for i in range(len(a))) == 4)

model.setObjective(grb.quicksum(v[i] * x[i] for i in range(len(a))))

model.ModelSense = grb.GRB.MAXIMIZE
model.optimize()


x_new = {i: x_i.x for i, x_i in x.items()}

alphas = {}

while True:
    # Find cover for that x_bar
    cover, zeta = getCover(weight=a, budget=budget, x_bar=x_new)
    c = len(cover)
    
    if zeta >= 1:   # Because x* satisfies all cover inequalities at this point.
        break

    values = dict(enumerate(a)) 

    # Sequential Lifting:
    # Choose an ordering
    ordering = list(set(range(n)) - set(alphas.keys()))

    # Permute the ordering and lift to get more inequalities
    permuted_orderings = permutations(ordering, len(ordering))
    
    for ii, permuted_ordering in enumerate(permuted_orderings):
        if ii > 5: break
        
        # Turn the tuple of permuted ordering into indexable list
        permuted_ordering = list(permuted_ordering)

        # Update alphas from last iteration
        new_alphas = alphas
        
        # Stop if t == r meaning alpha for the last index of the ordering is done.
        while len(permuted_ordering) != 0:  
            
            # Choose an index, t
            t = permuted_ordering.pop()
            
            # Perform Lifting
            alpha_t = lift(cover, new_alphas, t, values, budget)

            # Add alpha_t to the alphas if greater than 0
            if alpha_t:
                new_alphas[t] = alpha_t
                
            # print(new_alphas)
            # Put the new inequality in a set
            na = {'alphas': new_alphas.copy(), 'cover': cover}
            ineqs.add(HashableDict(na))
            
        fd_ineqs.add(HashableDict(na))

    # Add the new inequalities to the master problem and solve it.
    for ineq in ineqs:
        model.addConstr(grb.quicksum(ineq_i * x[i] for i, ineq_i in ineq['alphas'].items()) <= len(ineq['cover']) - 1)
    model.update()
    model.optimize()
    x_new = {i: x[i].x for i in x}
    
            
# print(ineqs)
# print()
# print(fd_ineqs)
solution = {i: x_i.x for i, x_i in x.items() if x_i.x > 0.5}
print(solution, model.ObjVal)
    

{0: 1.0, 24: 1.0, 61: 1.0, 62: 1.0} 391.3694032081638


In [20]:
gurobiKP(values=v, weights=a, budget=budget)

({0, 24, 61, 62}, 391.3694032081638)